In [ ]:
import requests

## Deployed Stack

Once you build your stack using the No-Code tool platform, you can deploy your app.

After deploying the app, use the deployed URL. If your app contains any input variables or external data used for retrieval-augmented generation (RAG), add the file path within your project directory.

In [ ]:
DEPLOYED_URL = "https://chat-with-pdf-e69a5be9.stack.app.aiplanet.com"
PDF_PATH = "sample.pdf"
QUESTION = "what is the total cooking time?"

## Get your API Key

Get your API key from here: [https://app.aiplanet.com/account/api-keys](https://app.aiplanet.com/account/api-keys)

In [ ]:
from getpass import getpass

API_URL_PREFIX = f"{DEPLOYED_URL}/api/v1/"
API_KEY = getpass("Enter Your GenAI stack API key: ") 

Enter Your GenAI stack API key: ··········


Documentation: [GenAI Stack API docs](https://docs.aiplanet.com/rest-apis/genai-stack-rest-apis)

In [ ]:
""" Defining header """
headers = {"Authorization": f"Bearer {API_KEY}"}

In [ ]:
api_url = API_URL_PREFIX + "apps"
response = requests.get(api_url, json={}, headers=headers)

In [ ]:
response.json()

In [ ]:
FLOW_SERVER_APP_ID = response.json()['id']

In [ ]:
""" Creating A New Chat Session """
api_url = API_URL_PREFIX + FLOW_SERVER_APP_ID + "/chat-sessions"
response = requests.post(api_url, json={"title":"Testing Chat Session"}, headers=headers)

In [ ]:
response.json()

{'id': 'eb2e7f01-5102-4460-83fb-2a304970f2b5',
 'title': 'Testing Chat Session',
 'data': None}

In [ ]:
CHAT_SESSION_ID1 = response.json()['id']

In [ ]:
api_url = API_URL_PREFIX + FLOW_SERVER_APP_ID + f"/chat-sessions/{CHAT_SESSION_ID1}/uploads"
form_data = {
    "input_key":"file_path"
}

# sample2 pdf is a document on cooking recipies
file = {'file': ("sample.pdf", open(f'{PDF_PATH}','rb'), 'application/pdf')}
response = requests.post(api_url, data=form_data, headers=headers, files=file)
file['file'][1].close()

In [ ]:
response.json()

{'id': 'c61b4bf4-7d97-48d5-883d-76f2b85fba80',
 'file': '/mnt/models/files/75579469-5cd2-47ec-a1e7-687f22d978ee/9090ea0d-a516-4951-8d78-8035081e60f4.pdf',
 'url': '',
 'text': '',
 'chat_session_id': 'eb2e7f01-5102-4460-83fb-2a304970f2b5',
 'created_at': '2024-06-04T09:04:56.985112',
 'meta_data': {'filename': 'sample.pdf', 'file_size': 133867},
 'data_pipeline_status': 'queued',
 'data_pipeline': {'id': 'f13398a4-9c78-484e-8a13-5d736463c986',
  'input_key': 'file_path',
  'meta_data': {'fileTypes': ['pdf'], 'suffixes': ['.pdf']},
  'created_at': '2024-05-28T17:49:03.645192'}}

In [ ]:
ID = response.json()['id']
CHAT_SESSION_ID2 = response.json()['chat_session_id']

In [ ]:
api_url = API_URL_PREFIX + FLOW_SERVER_APP_ID + f"/chat-sessions/{CHAT_SESSION_ID2}/uploads/{ID}"
response = requests.get(api_url, json={}, headers=headers)

In [ ]:
response.json()

{'id': 'c61b4bf4-7d97-48d5-883d-76f2b85fba80',
 'file': '/mnt/models/files/75579469-5cd2-47ec-a1e7-687f22d978ee/9090ea0d-a516-4951-8d78-8035081e60f4.pdf',
 'url': '',
 'text': '',
 'chat_session_id': 'eb2e7f01-5102-4460-83fb-2a304970f2b5',
 'created_at': '2024-06-04T09:04:56.985112',
 'meta_data': {'filename': 'sample.pdf', 'file_size': 133867},
 'data_pipeline_status': 'running',
 'data_pipeline': {'id': 'f13398a4-9c78-484e-8a13-5d736463c986',
  'input_key': 'file_path',
  'meta_data': {'fileTypes': ['pdf'], 'suffixes': ['.pdf']},
  'created_at': '2024-05-28T17:49:03.645192'}}

In [ ]:
CHAT_SESSION_ID3 = response.json()['chat_session_id']

In [ ]:
api_url = API_URL_PREFIX + FLOW_SERVER_APP_ID + f"/chat-sessions/{CHAT_SESSION_ID3}"
data={
    "chatKey": "question",
    "inputs":{
        "question":f"{QUESTION}"
    }
}

response = requests.post(api_url, json=data, headers=headers)

In [ ]:
text_response = response.text

In [ ]:
import json

response_objs = []
start = 0
for index, character in enumerate(text_response):
  if text_response[index] == "{" and text_response[index:index+10] == '{"is_bot":':
    start = index
  elif text_response[index] == "}" and text_response[index-11:index+1] == '"files": []}':
    response_objs.append(json.loads(text_response[start:index+1]))

In [ ]:
response_objs[-1].get("message")

'The total cooking time is 20 minutes.'